In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import scipy
import sys
import os
import pickle
import librosa
import librosa.display
from IPython.display import Audio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import keras
from keras.models import Sequential

In [ ]:
audio_data = '../input/gtzan-dataset-music-genre-classification/Data/genres_original/classical/classical.00026.wav'
data , sr = librosa.load(audio_data)
print(type(data), type(sr))

In [ ]:
librosa.load(audio_data, sr=45600)

Playing audio

In [ ]:
import IPython
IPython.display.Audio(data,rate=sr)

## Visualization

In [ ]:
plt.figure(figsize=(14,6))
librosa.display.waveplot(data)
plt.show()

In [ ]:
X = librosa.stft(data)
amp = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(14,6))
librosa.display.specshow(amp,sr=sr,x_axis='time',y_axis='hz')
plt.colorbar()

In [ ]:
plt.figure(figsize=(14, 6))
librosa.display.specshow(amp, sr=sr, x_axis='time', y_axis='log')
plt.colorbar()

In [ ]:
from sklearn.preprocessing import normalize

spectral_rolloff = librosa.feature.spectral_rolloff(data+0.01, sr=sr)[0]
plt.figure(figsize=(14, 6))
librosa.display.waveplot(data, sr=sr, alpha=0.4)

In [ ]:
plt.figure(figsize=(14, 6))
librosa.display.waveplot(data, sr=sr)

### Zero crossing

In [ ]:
n_0 = 9000
n_1 = 9100

zc = librosa.zero_crossings(data[n_0:n_1], pad=False)
print("The number of zero-crossings is :",sum(zc))

plt.figure(figsize=(14,6))
plt.plot(data[n_0:n_1])
plt.grid()

### Classification

In [ ]:
df1 = pd.read_csv('../input/gtzan-dataset-music-genre-classification/Data/features_3_sec.csv')
df1.head()

In [ ]:
print(df1.shape)
print(df1.dtypes)

In [ ]:
df1 = df1.drop(labels='filename',axis=1)

In [ ]:
genre_list = df1.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)
print(y)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(np.array(df1.iloc[:, :-1], dtype = float))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

## Neural networks

In [ ]:
m = Sequential()
m.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
m.add(layers.Dense(128, activation='relu'))
m.add(layers.Dense(64, activation='relu'))
m.add(layers.Dense(10, activation='softmax'))
m.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
classifier = m.fit(X_train,y_train,epochs=100,batch_size=128)

### Accuracy

In [ ]:
loss, acc  = m.evaluate(X_test, y_test, batch_size=128)
print('Loss = ',loss,'Accuracy = ',acc)


### Using validation set

In [ ]:
history = m.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=50,batch_size=32)

In [ ]:
fig,axs = plt.subplots(2)
axs[0].plot(history.history["accuracy"],label="train accuracy")
axs[0].plot(history.history["val_accuracy"],label="test accuracy")
axs[0].set_ylabel("Accuracy")
axs[0].legend(loc='lower right')
axs[0].set_title("Accuracy eval")
    
axs[1].plot(history.history["loss"],label="train error")
axs[1].plot(history.history["val_loss"],label="test error")
axs[1].set_ylabel("Error")
axs[1].set_xlabel("Epoch")
axs[1].legend(loc='upper right')
plt.show()

## CNN